In [0]:
process_or_create_table("reviews_without_text_table", None, summary=True, delete=False)

building reviews_without_text_table from existing table files
+------------+
|record_count|
+------------+
|     6990280|
+------------+

+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+
|             review_id|           business_id|               user_id|cool|funny|useful|               date|stars|
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+
|6jtUDueEHRG2rbAJ75bTLA|dJPTV6ilP1FhayMIyVMN2A|e-lnkltoLKcVtXvipOFIDg|   0|    0|     0|2020-01-03 00:22:30|  3.0|
|O-ZTyisaO1fRIyfadT3ssA|h8h1QvEQruTkBI50NlWsyA|Hco0gs1-5c07rHCDgxpfNg|   0|    0|     0|2020-01-31 12:10:05|  5.0|
|Q6aYY1c0ENTkm-yvuZr-1w|36NolK-R-VXqKMh7DiNSWw|8LFk1qnvM3SmxtkCh8UkGQ|   0|    0|     1|2019-07-29 23:52:36|  5.0|
|3j2US9m_3GyEcbYRzzc4gA|kOOiEcy0UaWLwCPgZ95nmg|iPkdNziqnhNukjnmsrSzmg|   1|    1|     7|2010-06-19 12:52:48|  2.0|
|KS9COVn1Fq2TKYsofgU3AA|8gY3LGFUMIQubovGbSd7Nw|JGJ_Vx1GK4

In [0]:
df_ordered_reviews = spark.sql("""
SELECT *, 
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS review_order
FROM reviews_without_text_table 
ORDER BY user_id, review_order
""")
print(f"review count:{df_ordered_reviews.count()}")
df_ordered_reviews.show(200, truncate=False)
df_ordered_reviews.createOrReplaceTempView("ordered_reviews")

review count:6990280
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+
|review_id             |business_id           |user_id               |cool|funny|useful|date               |stars|review_order|
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+
|rJ3CASyRfG-7ZviCBFCJQg|f19eLfhXqR47Ct8Hz2y_pA|---1lKK3aKOuomHnwAkAow|0   |0    |0     |2018-12-19 22:26:22|5.0  |1           |
|f-M5V-LeQKIy3U6vQ_FfYQ|pgO-fORYt4nb5Tj0x1F_aQ|---2PmXbF47D870stH1jqA|0   |0    |0     |2012-10-24 13:33:39|5.0  |1           |
|hepOKRE2O5gXDWLKkAjn5w|1An4DxtMmvvSe0HX4viRCA|---2PmXbF47D870stH1jqA|1   |0    |1     |2012-10-28 17:16:13|5.0  |2           |
|PtiOktOk5COHoNjc6K4gcw|eR7ieJD12PUzsYrP8fw6rQ|---2PmXbF47D870stH1jqA|0   |0    |0     |2012-11-02 00:30:24|5.0  |3           |
|xSRDnEBpog6z0IwltFEDxg|HpWi2CRJlxVCYKd8kS0X-A|---2PmXbF47D870stH1jqA|2   |0    |5 

In [0]:
df_useful = spark.sql("""
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS review_order,
       RANK() OVER (PARTITION BY user_id ORDER BY useful) AS rank,
       DENSE_RANK() OVER (PARTITION BY user_id ORDER BY useful) AS dense_rank,
       PERCENT_RANK() OVER (PARTITION BY user_id ORDER BY useful) AS percent_rank,
       NTILE(4) OVER (PARTITION BY user_id ORDER BY useful) AS quartile
FROM reviews_without_text_table
ORDER BY user_id, useful
""")
print(f"review count:{df_useful.count()}")
df_useful.show(100, truncate=False)

review count:6990280
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+----+----------+--------------------+--------+
|review_id             |business_id           |user_id               |cool|funny|useful|date               |stars|review_order|rank|dense_rank|percent_rank        |quartile|
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+----+----------+--------------------+--------+
|rJ3CASyRfG-7ZviCBFCJQg|f19eLfhXqR47Ct8Hz2y_pA|---1lKK3aKOuomHnwAkAow|0   |0    |0     |2018-12-19 22:26:22|5.0  |1           |1   |1         |0.0                 |1       |
|xJuVVh0wspQlCPgTcbbiIg|hKameFsaXh9g8WQbv593UA|---2PmXbF47D870stH1jqA|0   |0    |0     |2014-10-28 14:38:58|5.0  |13          |1   |1         |0.0                 |1       |
|c4epF4UIlXj1gvDsBa6cIA|1An4DxtMmvvSe0HX4viRCA|---2PmXbF47D870stH1jqA|0   |0    |0     |2014-11-02 13:47:48|5

In [0]:
df_ordered_reviews = spark.sql("""
SELECT *, 
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS review_order,
       AVG(stars) OVER (PARTITION BY business_id) AS business_avg
FROM reviews_without_text_table 
ORDER BY user_id, review_order
""")
print(f"review count:{df_ordered_reviews.count()}")
df_ordered_reviews.show(200, truncate=False)
df_ordered_reviews.createOrReplaceTempView("ordered_reviews")


review count:6990280
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+------------------+
|review_id             |business_id           |user_id               |cool|funny|useful|date               |stars|review_order|business_avg      |
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+------------------+
|rJ3CASyRfG-7ZviCBFCJQg|f19eLfhXqR47Ct8Hz2y_pA|---1lKK3aKOuomHnwAkAow|0   |0    |0     |2018-12-19 22:26:22|5.0  |1           |4.849056603773585 |
|f-M5V-LeQKIy3U6vQ_FfYQ|pgO-fORYt4nb5Tj0x1F_aQ|---2PmXbF47D870stH1jqA|0   |0    |0     |2012-10-24 13:33:39|5.0  |1           |3.4251497005988023|
|hepOKRE2O5gXDWLKkAjn5w|1An4DxtMmvvSe0HX4viRCA|---2PmXbF47D870stH1jqA|1   |0    |1     |2012-10-28 17:16:13|5.0  |2           |3.5365853658536586|
|PtiOktOk5COHoNjc6K4gcw|eR7ieJD12PUzsYrP8fw6rQ|---2PmXbF47D870stH1jqA|0   |0    |0     |2012-11-0

In [0]:
df_ordered_reviews = spark.sql("""
SELECT *, 
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS review_order,
       AVG(stars) OVER (PARTITION BY business_id ORDER BY date 
                        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS business_avg
FROM reviews_without_text_table 
ORDER BY user_id, review_order
""")
print(f"review count:{df_ordered_reviews.count()}")
df_ordered_reviews.show(200, truncate=False)
df_ordered_reviews.createOrReplaceTempView("ordered_reviews")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2402765643583551> in <module>
      1 # Add your query here (Start with Step 4a)
----> 2 df_ordered_reviews = spark.sql("""
      3 SELECT *,
      4        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS review_order,
      5        AVG(stars) OVER (PARTITION BY business_id ORDER BY date 

/databricks/spark/python/pyspark/sql/session.py in sql(self, sqlQuery)
    775         [Row(f1=1, f2='row1'), Row(f1=2, f2='row2'), Row(f1=3, f2='row3')]
    776         """
--> 777         return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
    778 
    779     def table(self, tableName):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305 

In [0]:
 df_ordered_reviews = spark.sql("""
SELECT *, 
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS review_order,
       AVG(stars) OVER (PARTITION BY business_id ORDER BY date 
                        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS business_avg,
       COUNT(review_id) OVER (PARTITION BY business_id ORDER BY date
                              ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS prior_reviews
FROM reviews_without_text_table 
ORDER BY user_id, review_order
""")
print(f"review count:{df_ordered_reviews.count()}")
df_ordered_reviews.show(200, truncate=False)
df_ordered_reviews.createOrReplaceTempView("ordered_reviews")



review count:6990280
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+------------------+-------------+
|review_id             |business_id           |user_id               |cool|funny|useful|date               |stars|review_order|business_avg      |prior_reviews|
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+------------------+-------------+
|rJ3CASyRfG-7ZviCBFCJQg|f19eLfhXqR47Ct8Hz2y_pA|---1lKK3aKOuomHnwAkAow|0   |0    |0     |2018-12-19 22:26:22|5.0  |1           |4.931034482758621 |29           |
|f-M5V-LeQKIy3U6vQ_FfYQ|pgO-fORYt4nb5Tj0x1F_aQ|---2PmXbF47D870stH1jqA|0   |0    |0     |2012-10-24 13:33:39|5.0  |1           |4.125             |8            |
|hepOKRE2O5gXDWLKkAjn5w|1An4DxtMmvvSe0HX4viRCA|---2PmXbF47D870stH1jqA|1   |0    |1     |2012-10-28 17:16:13|5.0  |2           |2.5               |2            |
|PtiOktOk5COH

In [0]:
process_or_create_table("ordered_reviews_table", "df_ordered_reviews", summary=True, delete=False)

Saving the df_ordered_reviews DataFrame as table: ordered_reviews_table
+------------+
|record_count|
+------------+
|     6990280|
+------------+

+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+------------------+-------------+
|             review_id|           business_id|               user_id|cool|funny|useful|               date|stars|review_order|      business_avg|prior_reviews|
+----------------------+----------------------+----------------------+----+-----+------+-------------------+-----+------------+------------------+-------------+
|rJ3CASyRfG-7ZviCBFCJQg|f19eLfhXqR47Ct8Hz2y_pA|---1lKK3aKOuomHnwAkAow|   0|    0|     0|2018-12-19 22:26:22|  5.0|           1| 4.931034482758621|           29|
|f-M5V-LeQKIy3U6vQ_FfYQ|pgO-fORYt4nb5Tj0x1F_aQ|---2PmXbF47D870stH1jqA|   0|    0|     0|2012-10-24 13:33:39|  5.0|           1|             4.125|            8|
|hepOKRE2O5gXDWLKkAjn5w|1An4DxtMmvvSe0HX4viRCA|